In [1]:
import pyspark
from delta.pip_utils import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')  \
    .config("spark.hadoop.fs.s3a.endpoint", "localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

my_packages = ["org.apache.hadoop:hadoop-aws:3.3.1"]
spark = configure_spark_with_delta_pip(builder, extra_packages=my_packages).getOrCreate()

23/06/22 15:27:30 WARN Utils: Your hostname, Noahs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.32 instead (on interface en0)
23/06/22 15:27:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/nprince/.ivy2/cache
The jars for the packages stored in: /Users/nprince/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ca926f6-bae3-4b3f-809d-4fca25e4d572;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 234ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hado

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [20]:
spark.read.format("delta").load("s3a://delta/silver/test10").createOrReplaceTempView("test")
spark.read.format("delta").load("s3a://delta/bronze/iot_poc_v17").createOrReplaceTempView("iot_poc_v1")


In [19]:
spark.sql("""\
SELECT beacon_report.received_timestamp / 1000 one, from_unixtime(beacon_report.received_timestamp / 1000, 'yyyy-MM-dd HH:mm:ss.SSSSSS') from iot_poc_v1 limit 1
""").toPandas().head()

,one,"from_unixtime((beacon_report.received_timestamp / 1000), yyyy-MM-dd HH:mm:ss.SSSSSS)"
0,1677312824.150000,2023-02-25 02:13:44.000000


In [26]:
spark.sql("""\
SELECT beacon.received_timestamp, beacon.pub_key
from test
where beacon.received_timestamp >= cast('2023-02-25 02:11:46' as timestamp) and 
      beacon.received_timestamp <=cast('2023-02-25 02:11:48' as timestamp) 
      and selected = true 
limit 5
""").show(truncate=False)

+-----------------------+---------------------------------------------------+
|received_timestamp     |pub_key                                            |
+-----------------------+---------------------------------------------------+
|2023-02-25 02:11:46.891|11TR6JvxBD6TuSya21WcGMPacAVeo5rMyP3RjHfpF7L1LmXokEz|
|2023-02-25 02:11:46.036|11Jwii1aKzwxF1ipkncC8mjhyYYPLdLXWfb7hz6EdcG2g3Bezj8|
|2023-02-25 02:11:46.036|11Jwii1aKzwxF1ipkncC8mjhyYYPLdLXWfb7hz6EdcG2g3Bezj8|
|2023-02-25 02:11:46.036|11Jwii1aKzwxF1ipkncC8mjhyYYPLdLXWfb7hz6EdcG2g3Bezj8|
|2023-02-25 02:11:46.036|11Jwii1aKzwxF1ipkncC8mjhyYYPLdLXWfb7hz6EdcG2g3Bezj8|
+-----------------------+---------------------------------------------------+

